In [7]:
library(tidyverse)
library(magrittr)

# Pre-processing

This example uses the [population data](http://www.abs.gov.au/AUSSTATS/subscriber.nsf/log?openagent&32180ds0002_2017-18.xls&3218.0&Data%20Cubes&8BA68C878E440190CA2583C9000DF24E&0&2017-18&27.03.2019&Latest), which is in Excel format with gaps in the rows and columns, not to mention several rows of headers, some of which are merged.

![Population](/Misc/pre_process_abs_data.png)

The population data used in this example takes out the blanks, removes the extra header rows and makes some changes to the names of the headers. The data itself starts from cell `A10`.

In [67]:
pop_path <- paste(getwd(), "Data", "Population", "pop_2017-18_nsw.csv", sep = "/")
pop_path
stop_coords_path <- paste(getwd(), "Output", "final_stop_coords_df.csv", sep = "/")
stop_coords_path

[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Data/Population/pop_2017-18_nsw.csv"

[1] "C:/Users/Mactuary/OneDrive - UTS/36103/AT2/Output/final_stop_coords_df.csv"

In [68]:
pop_df <- read_csv(pop_path)
glimpse(pop_df)

Parsed with column specification:
cols(
  `LGA code` = col_double(),
  `Local Government Area` = col_character(),
  `2017` = col_double(),
  `2018` = col_double(),
  `2017-2018_change` = col_double(),
  `2017-2018_change_per` = col_double(),
  `Natural increase` = col_double(),
  `Net internal migration` = col_double(),
  `Net overseas migration` = col_double(),
  Area = col_double(),
  Density = col_double()
)


Observations: 129
Variables: 11
$ `LGA code`               <dbl> 10050, 10130, 10250, 10300, 10470, 10500, ...
$ `Local Government Area`  <chr> "Albury (C)", "Armidale Regional (A)", "Ba...
$ `2017`                   <dbl> 52989, 30617, 43480, 2343, 42806, 170266, ...
$ `2018`                   <dbl> 53767, 30707, 44208, 2340, 43206, 174378, ...
$ `2017-2018_change`       <dbl> 778, 90, 728, -3, 400, 4112, 231, 12, 40, ...
$ `2017-2018_change_per`   <dbl> 1.5, 0.3, 1.7, -0.1, 0.9, 2.4, 0.7, 0.1, 0...
$ `Natural increase`       <dbl> 288, 117, -44, 13, 198, 1358, -69, -18, 5,...
$ `Net internal migration` <dbl> 218, -374, 584, -22, 42, -1842, 222, -10, ...
$ `Net overseas migration` <dbl> 272, 347, 188, 6, 160, 4596, 78, 40, 36, 4...
$ Area                     <dbl> 305.9, 8620.7, 484.9, 21690.7, 3817.9, 49....
$ Density                  <dbl> 175.7, 3.6, 91.2, 0.1, 11.3, 3495.2, 5.5, ...


In [16]:
stop_coords_df <- read_csv(stop_coords_path)
glimpse(stop_coords_df)

Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  X1 = col_double(),
  stop_id = col_character(),
  stop_lon = col_double(),
  stop_lat = col_double(),
  lga_number = col_double(),
  total_lga = col_double(),
  LG_PLY_PID = col_double(),
  DT_CREATE = col_date(format = ""),
  DT_RETIRE = col_logical(),
  LGA_PID = col_character(),
  NSW_LGA_sh = col_date(format = ""),
  NSW_LGA__1 = col_logical(),
  NSW_LGA__2 = col_character(),
  NSW_LGA__3 = col_character(),
  NSW_LGA__4 = col_logical(),
  NSW_LGA__5 = col_double(),
  route_type = col_double(),
  num_service = col_double()
)


Observations: 70,869
Variables: 18
$ X1          <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, ...
$ stop_id     <chr> "228055", "228055", "228054", "228054", "287146", "2280...
$ stop_lon    <dbl> 151.6856, 151.6856, 151.6846, 151.6846, 148.0150, 151.6...
$ stop_lat    <dbl> -33.00361, -33.00361, -33.00619, -33.00619, -33.39237, ...
$ lga_number  <dbl> 35, 35, 35, 35, 28, 35, 35, 35, 35, 28, 35, 35, 35, 35,...
$ total_lga   <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...
$ LG_PLY_PID  <dbl> 2958, 2958, 2958, 2958, 2975, 2958, 2958, 2958, 2958, 2...
$ DT_CREATE   <date> 2017-09-12, 2017-09-12, 2017-09-12, 2017-09-12, 2017-0...
$ DT_RETIRE   <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,...
$ LGA_PID     <chr> "NSW206", "NSW206", "NSW206", "NSW206", "NSW265", "NSW2...
$ NSW_LGA_sh  <date> 2011-11-29, 2011-11-29, 2011-11-29, 2011-11-29, 2011-1...
$ NSW_LGA__1  <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,...
$ NSW_LGA__2  <ch

# Removing the Parentheses

The `Local Government Area` column contains the LGA name and the type, e.g. `"Albury (C)"`. Other data sets don't have the type. In order to join by the `Local Government Area`, it's necessary to remove the type information.

In [70]:
pop_df %>%
    select(`Local Government Area`) %>%
    head

Local Government Area
Albury (C)
Armidale Regional (A)
Ballina (A)
Balranald (A)
Bathurst Regional (A)
Bayside (A)


Using `gsub` and regex to remove any instances of parentheses and their contents.
The lga names are also converted to lower case for matching.

In [71]:
pop_lga <- pop_df %>% 
    mutate(lga = gsub("\\s*\\([^\\)]+\\)","", `Local Government Area`)) %>%
    mutate(lga = tolower(lga))
glimpse(pop_lga)

pop_lga %>%
    select(`Local Government Area`, lga) %>%
    head

Observations: 129
Variables: 12
$ `LGA code`               <dbl> 10050, 10130, 10250, 10300, 10470, 10500, ...
$ `Local Government Area`  <chr> "Albury (C)", "Armidale Regional (A)", "Ba...
$ `2017`                   <dbl> 52989, 30617, 43480, 2343, 42806, 170266, ...
$ `2018`                   <dbl> 53767, 30707, 44208, 2340, 43206, 174378, ...
$ `2017-2018_change`       <dbl> 778, 90, 728, -3, 400, 4112, 231, 12, 40, ...
$ `2017-2018_change_per`   <dbl> 1.5, 0.3, 1.7, -0.1, 0.9, 2.4, 0.7, 0.1, 0...
$ `Natural increase`       <dbl> 288, 117, -44, 13, 198, 1358, -69, -18, 5,...
$ `Net internal migration` <dbl> 218, -374, 584, -22, 42, -1842, 222, -10, ...
$ `Net overseas migration` <dbl> 272, 347, 188, 6, 160, 4596, 78, 40, 36, 4...
$ Area                     <dbl> 305.9, 8620.7, 484.9, 21690.7, 3817.9, 49....
$ Density                  <dbl> 175.7, 3.6, 91.2, 0.1, 11.3, 3495.2, 5.5, ...
$ lga                      <chr> "albury", "armidale regional", "ballina", ...


Local Government Area,lga
Albury (C),albury
Armidale Regional (A),armidale regional
Ballina (A),ballina
Balranald (A),balranald
Bathurst Regional (A),bathurst regional
Bayside (A),bayside


# Grouping the `stop_coords_df`

Changing the `NSW_LGA__3` values to lower case for matching.

In [56]:
stop_coords_df %<>%
    mutate(NSW_LGA__3 = tolower(NSW_LGA__3))
glimpse(stop_coords_df)

Observations: 70,869
Variables: 18
$ X1          <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, ...
$ stop_id     <chr> "228055", "228055", "228054", "228054", "287146", "2280...
$ stop_lon    <dbl> 151.6856, 151.6856, 151.6846, 151.6846, 148.0150, 151.6...
$ stop_lat    <dbl> -33.00361, -33.00361, -33.00619, -33.00619, -33.39237, ...
$ lga_number  <dbl> 35, 35, 35, 35, 28, 35, 35, 35, 35, 28, 35, 35, 35, 35,...
$ total_lga   <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...
$ LG_PLY_PID  <dbl> 2958, 2958, 2958, 2958, 2975, 2958, 2958, 2958, 2958, 2...
$ DT_CREATE   <date> 2017-09-12, 2017-09-12, 2017-09-12, 2017-09-12, 2017-0...
$ DT_RETIRE   <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,...
$ LGA_PID     <chr> "NSW206", "NSW206", "NSW206", "NSW206", "NSW265", "NSW2...
$ NSW_LGA_sh  <date> 2011-11-29, 2011-11-29, 2011-11-29, 2011-11-29, 2011-1...
$ NSW_LGA__1  <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,...
$ NSW_LGA__2  <ch

Grouping by `NSW_LGA__3` and `route_type` to aggregate (sum) over the `stop_id` while maintaining the information on the LGA and mode of transportation.

In [58]:
stop_coords_grouped <- stop_coords_df %>% 
    group_by(NSW_LGA__3, route_type) %>%
    summarize(num_service = sum(num_service)) %>%
    drop_na(NSW_LGA__3, route_type)
head(stop_coords_grouped)

NSW_LGA__3,route_type,num_service
albury,106,131
albury,204,14
albury,700,1624
armidale regional,106,62
armidale regional,204,10
armidale regional,700,113


# Merging the Data

The join keys are `NSW_LGA__3` and `lga` for the `stop_coords_df` and `seifa_df` tables respectively.

In [60]:
head(stop_coords_df)

X1,stop_id,stop_lon,stop_lat,lga_number,total_lga,LG_PLY_PID,DT_CREATE,DT_RETIRE,LGA_PID,NSW_LGA_sh,NSW_LGA__1,NSW_LGA__2,NSW_LGA__3,NSW_LGA__4,NSW_LGA__5,route_type,num_service
1,228055,151.6856,-33.00361,35,1,2958,2017-09-12,NA,NSW206,2011-11-29,NA,LAKE MACQUARIE CITY COUNCIL,lake macquarie,NA,1,700,3
2,228055,151.6856,-33.00361,35,1,2958,2017-09-12,NA,NSW206,2011-11-29,NA,LAKE MACQUARIE CITY COUNCIL,lake macquarie,NA,1,712,4
3,228054,151.6846,-33.00619,35,1,2958,2017-09-12,NA,NSW206,2011-11-29,NA,LAKE MACQUARIE CITY COUNCIL,lake macquarie,NA,1,700,3
4,228054,151.6846,-33.00619,35,1,2958,2017-09-12,NA,NSW206,2011-11-29,NA,LAKE MACQUARIE CITY COUNCIL,lake macquarie,NA,1,712,5
5,287146,148.0150,-33.39237,28,1,2975,2017-09-12,NA,NSW265,2011-11-29,NA,FORBES SHIRE COUNCIL,forbes,NA,1,700,3
6,228056,151.6855,-32.99985,35,1,2958,2017-09-12,NA,NSW206,2011-11-29,NA,LAKE MACQUARIE CITY COUNCIL,lake macquarie,NA,1,700,92


In [72]:
stop_pop_joined <- stop_coords_grouped %>%
    left_join(pop_lga, by = c("NSW_LGA__3" = "lga"))

head(stop_seifa_joined, 20)

NSW_LGA__3,route_type,num_service,LGA code,Local Government Area,2017,2018,2017-2018_change,2017-2018_change_per,Natural increase,Net internal migration,Net overseas migration,Area,Density
albury,106,131,10050,Albury (C),52989,53767,778,1.5,288,218,272,305.9,175.7
albury,204,14,10050,Albury (C),52989,53767,778,1.5,288,218,272,305.9,175.7
albury,700,1624,10050,Albury (C),52989,53767,778,1.5,288,218,272,305.9,175.7
armidale regional,106,62,10130,Armidale Regional (A),30617,30707,90,0.3,117,-374,347,8620.7,3.6
armidale regional,204,10,10130,Armidale Regional (A),30617,30707,90,0.3,117,-374,347,8620.7,3.6
armidale regional,700,113,10130,Armidale Regional (A),30617,30707,90,0.3,117,-374,347,8620.7,3.6
ballina,204,18,10250,Ballina (A),43480,44208,728,1.7,-44,584,188,484.9,91.2
ballina,700,1335,10250,Ballina (A),43480,44208,728,1.7,-44,584,188,484.9,91.2
balranald,204,4,10300,Balranald (A),2343,2340,-3,-0.1,13,-22,6,21690.7,0.1
bathurst regional,2,49,10470,Bathurst Regional (A),42806,43206,400,0.9,198,42,160,3817.9,11.3
